In [6]:
!nvidia-smi

Sun Sep 18 07:32:03 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [7]:
import json
token = {"username":"  ","key":"  "}#使用你自己的用户和密钥 Use your own username and key
with open('/content/kaggle.json', 'w') as file:
  json.dump(token, file)
!mkdir -p ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle config set -n path -v /content

- path is now set to: /content


In [8]:
!kaggle datasets download -d lzt2423654259/trashnet

 42% 17.0M/40.8M [00:00<00:00, 60.7MB/s]
100% 40.8M/40.8M [00:00<00:00, 119MB/s] 


In [9]:
!unzip '/content/datasets/lzt2423654259/trashnet/trashnet.zip'

Archive:  /content/datasets/lzt2423654259/trashnet/trashnet.zip
  inflating: dataset-resized/cardboard/cardboard1.jpg  
  inflating: dataset-resized/cardboard/cardboard10.jpg  
  inflating: dataset-resized/cardboard/cardboard100.jpg  
  inflating: dataset-resized/cardboard/cardboard101.jpg  
  inflating: dataset-resized/cardboard/cardboard102.jpg  
  inflating: dataset-resized/cardboard/cardboard103.jpg  
  inflating: dataset-resized/cardboard/cardboard104.jpg  
  inflating: dataset-resized/cardboard/cardboard105.jpg  
  inflating: dataset-resized/cardboard/cardboard106.jpg  
  inflating: dataset-resized/cardboard/cardboard107.jpg  
  inflating: dataset-resized/cardboard/cardboard108.jpg  
  inflating: dataset-resized/cardboard/cardboard109.jpg  
  inflating: dataset-resized/cardboard/cardboard11.jpg  
  inflating: dataset-resized/cardboard/cardboard110.jpg  
  inflating: dataset-resized/cardboard/cardboard111.jpg  
  inflating: dataset-resized/cardboard/cardboard112.jpg  
  inflating:

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import DataLoader
from torchvision import transforms, models, datasets
import time
from torch.autograd import Variable, Function
import torch as t 
from torch.utils import data 
from torchvision import transforms as T 
from PIL import Image  
from torchvision.datasets import ImageFolder
import random
import os
from torch.optim import lr_scheduler

In [11]:
class Trash(data.Dataset):
    def __init__(self, root, transform = True, train = True, seed = 1):
        self.labels = {'paper':0, 'glass':1, 'plastic':2, 'cardboard':3, 'trash':4, 'metal':5}
        dirs = os.listdir(root)#返回文件列表
        imgs = []#建立空列表
        for d in dirs:
            path = os.path.join(root, d)
            for img in os.listdir(path):
                imgs.append(os.path.join(path, img))

        random.seed(seed)
        random.shuffle(imgs)
        num_train = int(0.7 * len(imgs))#训练集个数
        if train:
            # training
            self.imgs = imgs[:num_train]#设置训练集
        else:
            # validation
            self.imgs = imgs[num_train:]#
        #random.shuffle(imgs)
        if transform is True:
            normalize = T.Normalize(mean = [0.5, 0.5, 0.5], std = [0.5, 0.5, 0.5])
            if not train:
                # 验证集
                self.transform = T.Compose([
                    T.Resize(224),#最短边224像素
                    T.CenterCrop(224),#中心裁剪
                    T.ToTensor(),#转成tensor
                    normalize#归一化
                ])
            else:
                # 训练集，做数据增广
                self.transform = T.Compose([
                    T.Resize(224),
                    T.RandomResizedCrop(224),
                    T.RandomHorizontalFlip(),#按概率水平翻转
                    T.ToTensor(),
                    normalize
                ])
        else:
            self.transform = transform

    def __getitem__(self, index):
        img_path = self.imgs[index]
        label = self.labels[img_path.split('/')[-2]]
        data = Image.open(img_path)
        data = self.transform(data)
        return data, label

    def __len__(self):
        return len(self.imgs)

In [12]:
class basic_block(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(basic_block, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=1, stride=1, padding=0)
        self.bn2 = nn.BatchNorm2d(out_channels)

    def forward(self, x):
        y = F.relu(self.conv1(x))
        y = self.bn1(y)
        y = self.conv2(y)
        y = self.bn2(y)
        return F.relu(x + y)

class basic_block2(nn.Module):
    def __init__(self,in_channels,out_channels):# 3-64 64-128 128-256
        super(basic_block2, self).__init__()
        self.conv1 = nn.Conv2d(in_channels,out_channels,kernel_size=1,stride=2,padding=0)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(in_channels,out_channels,kernel_size=3,stride=2,padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv3 = nn.Conv2d(out_channels,out_channels,kernel_size=1,stride=1,padding=0)
        self.bn3 = nn.BatchNorm2d(out_channels)
    def forward(self, x):
        z = self.conv1(x)
        z = self.bn1(z)
        y = F.relu(self.bn2(self.conv2(x)))
        y = self.conv3(y)
        y = self.bn3(y)
        return F.relu(y+z)

class basic_block3(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(basic_block3, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.drcov = DRConv2d(out_channels, out_channels, kernel_size=3, region_num=8)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=1, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)

    def forward(self, x):
        
        y = F.relu(self.conv1(x))
        
        y = self.bn1(y)
        y = self.drcov(y)
        
        y = self.conv2(y)
        
        y = self.bn2(y)
        return F.relu(x + y)

class asign_index(torch.autograd.Function):
    @staticmethod
    def forward(ctx, kernel, guide_feature):
        ctx.save_for_backward(kernel, guide_feature)
        guide_mask = torch.zeros_like(guide_feature).scatter_(1, guide_feature.argmax(dim=1, keepdim=True),
                                                              1).unsqueeze(2)  # B x 3 x 1 x 25 x 25
        return torch.sum(kernel * guide_mask, dim=1)

    @staticmethod
    def backward(ctx, grad_output):
        kernel, guide_feature = ctx.saved_tensors
        guide_mask = torch.zeros_like(guide_feature).scatter_(1, guide_feature.argmax(dim=1, keepdim=True),
                                                              1).unsqueeze(2)  # B x 3 x 1 x 25 x 25
        grad_kernel = grad_output.clone().unsqueeze(1) * guide_mask  # B x 3 x 256 x 25 x 25
        grad_guide = grad_output.clone().unsqueeze(1) * kernel  # B x 3 x 256 x 25 x 25
        grad_guide = grad_guide.sum(dim=2)  # B x 3 x 25 x 25
        softmax = F.softmax(guide_feature, 1)  # B x 3 x 25 x 25
        grad_guide = softmax * (grad_guide - (softmax * grad_guide).sum(dim=1, keepdim=True))  # B x 3 x 25 x 25
        return grad_kernel, grad_guide


def xcorr_slow(x, kernel, kwargs):
    """for loop to calculate cross correlation
    """
    batch = x.size()[0]
    out = []
    for i in range(batch):
        px = x[i]
        pk = kernel[i]
        px = px.view(1, px.size()[0], px.size()[1], px.size()[2])
        pk = pk.view(-1, px.size()[1], pk.size()[1], pk.size()[2])
        po = F.conv2d(px, pk, **kwargs)
        out.append(po)
    out = torch.cat(out, 0)
    return out


def xcorr_fast(x, kernel, kwargs):
    """group conv2d to calculate cross correlation
    """
    batch = kernel.size()[0]
    pk = kernel.view(-1, x.size()[1], kernel.size()[2], kernel.size()[3])
    px = x.view(1, -1, x.size()[2], x.size()[3])
    po = F.conv2d(px, pk, **kwargs, groups=batch)
    po = po.view(batch, -1, po.size()[2], po.size()[3])
    return po


class Corr(Function):
    @staticmethod
    def symbolic(g, x, kernel, groups):
        return g.op("Corr", x, kernel, groups_i=groups)

    @staticmethod
    def forward(self, x, kernel, groups, kwargs):
        """group conv2d to calculate cross correlation
        """
        batch = x.size(0)
        channel = x.size(1)
        x = x.view(1, -1, x.size(2), x.size(3))
        kernel = kernel.view(-1, channel // groups, kernel.size(2), kernel.size(3))
        out = F.conv2d(x, kernel, **kwargs, groups=groups * batch)
        out = out.view(batch, -1, out.size(2), out.size(3))
        return out


class Correlation(nn.Module):
    use_slow = True

    def __init__(self, use_slow=None):
        super(Correlation, self).__init__()
        if use_slow is not None:
            self.use_slow = use_slow
        else:
            self.use_slow = Correlation.use_slow

    def extra_repr(self):
        if self.use_slow: return "xcorr_slow"
        return "xcorr_fast"

    def forward(self, x, kernel, **kwargs):
        if self.training:
            if self.use_slow:
                return xcorr_slow(x, kernel, kwargs)
            else:
                return xcorr_fast(x, kernel, kwargs)
        else:
            return Corr.apply(x, kernel, 1, kwargs)


class DRConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, region_num=8, **kwargs):
        super(DRConv2d, self).__init__()
        self.region_num = region_num

        self.conv_kernel = nn.Sequential(
            nn.AdaptiveAvgPool2d((kernel_size, kernel_size)),
            nn.Conv2d(in_channels, region_num * region_num, kernel_size=1),
            nn.Sigmoid(),
            nn.Conv2d(region_num * region_num, region_num * in_channels * out_channels, kernel_size=1,
                      groups=region_num)
        )
        self.conv_guide = nn.Conv2d(in_channels, region_num, kernel_size=kernel_size, **kwargs)

        self.corr = Correlation(use_slow=False)
        self.kwargs = kwargs
        self.asign_index = asign_index.apply

    def forward(self, input):
        kernel = self.conv_kernel(input)
        kernel = kernel.view(kernel.size(0), -1, kernel.size(2), kernel.size(3))  # B x (r*in*out) x W X H
        output = self.corr(input, kernel, **self.kwargs)  # B x (r*out) x W x H
        output = output.view(output.size(0), self.region_num, -1, output.size(2), output.size(3))  # B x r x out x W x H
        guide_feature = self.conv_guide(input)
        output = self.asign_index(output, guide_feature)
        return output

In [13]:
class resnet_test(nn.Module):
    '''按照网络结构图直接连接，确定好通道数量就可以'''

    def __init__(self):
        super(resnet_test, self).__init__()
        self.resn1 = basic_block2(3, 64)
        self.resn2 = basic_block3(64, 64)
        self.resn3 = basic_block3(64, 64)
        self.resn4 = basic_block2(64, 128)
        self.resn5 = basic_block3(128, 128)
        self.resn6 = basic_block3(128, 128)
        self.resn7 = basic_block3(128, 128)
        self.resn8 = basic_block2(128, 256)
        self.resn9 = basic_block(256, 256)
        self.resn10 = basic_block(256, 256)
        self.resn11 = basic_block(256, 256)
        self.resn12 = basic_block(256, 256)
        self.resn13 = basic_block(256, 256)
        self.resn14 = basic_block(256, 256)
        self.resn15 = basic_block2(256, 512)
        self.resn16 = basic_block(512, 512)
        self.resn17 = basic_block(512, 512)
        self.avgpool = nn.AdaptiveAvgPool2d(output_size=(1, 1))
        self.fc = nn.Linear(512, 6)

    def forward(self, x):
        x = self.resn1(x)
        x = self.resn2(x)
        x = self.resn3(x)
        x = self.resn4(x)
        x = self.resn5(x)
        x = self.resn6(x)
        x = self.resn7(x)
        x = self.resn8(x)
        x = self.resn9(x)
        x = self.resn10(x)
        x = self.resn11(x)
        x = self.resn12(x)
        x = self.resn13(x)
        x = self.resn14(x)
        x = self.resn15(x)
        x = self.resn16(x)
        x = self.resn17(x)
        x = self.avgpool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)
        return x


In [14]:
def main():  # dataset_1 dataset-resized
    root = '/content/dataset-resized'
    train_data = Trash(root, transform=True, train=True)
    val_data = Trash(root, train=False)
    train_loader = DataLoader(
        train_data,
        batch_size=64,
        shuffle=True,
        num_workers=0
    )
    val_loader = DataLoader(
        val_data,
        batch_size=64,
        shuffle=False,
        num_workers=0
    )
    model = resnet_test()
    print(model)
    resnet34 = models.resnet34(pretrained=True)
    # 读取参数
    resnet34_dict = resnet34.state_dict()
    model_dict = model.state_dict()
    # 将pretrained_dict里不属于model_dict的键剔除掉
    resnet34__dict = {k: v for k, v in resnet34_dict.items() if (k in model_dict and 'fc' not in k)}
    # 更新现有的model_dict
    model_dict.update(resnet34__dict)
    # 加载我们真正需要的state_dict
    model.load_state_dict(model_dict)
    # state_dict = torch.load('/content/drive/MyDrive/myCal/task_1/CNN_T.pkl')
    # model.load_state_dict(state_dict)
    device = torch.device('cuda:0')
    model = model.to(device)
    criteon = nn.CrossEntropyLoss().to(device)#交叉熵损失
    optimizer = optim.Adam(model.parameters(), lr=1e-2)#优化器
    scheduler = lr_scheduler.MultiStepLR(optimizer, milestones=[50,80], gamma=0.1)
    for epoch in range(100):
        model.train()
        since = time.time()
        for x, label in train_loader:
            # [b, 3, 32, 32]
            # [b]
            x, label = x.to(device), label.to(device)#传给GPU
            logits = model(x)#返回值
            # logits: [b, 10]
            # label:  [b]
            # loss: tensor scalar
            loss = criteon(logits, label)#计算损失
            # backprop
            optimizer.zero_grad()#梯度清零
            loss.backward()
            optimizer.step()
        print('Epoch: ', epoch, '训练集 loss: ', loss.item())#item是转成数字
        model.eval()
        with torch.no_grad():
            # 测试集
            total_correct = 0
            total_num = 0
            for x, label in val_loader:
                x, label = x.to(device), label.to(device)
                logits = model(x)
                loss = criteon(logits, label)
                pred = logits.argmax(dim=1)#获得每行最大值列号
                correct = torch.eq(pred, label).float().sum().item()
                total_correct += correct
                total_num += x.size(0)
            acc = total_correct / total_num
            time_elapsed = time.time() - since
            print('Epoch: ', epoch, '测试集 loss: ', loss.item())#item是转成数字
            print('Epoch: ',epoch,' Training complete in {:.0f}min {:.0f}seconds'.format(time_elapsed // 60, time_elapsed % 60))
            timeCost = 'Training time {:.0f}min {:.0f}seconds'.format(time_elapsed // 60, time_elapsed % 60)
            print('Epoch: ',epoch, ' test acc: ', acc)     
        scheduler.step()
    # torch.save(model.state_dict(), '/content/drive/MyDrive/myCal/task_1/CNN_T.pkl')



In [15]:
if __name__ == '__main__':
    main()

resnet_test(
  (resn1): basic_block2(
    (conv1): Conv2d(3, 64, kernel_size=(1, 1), stride=(2, 2))
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv3): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
    (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (resn2): basic_block3(
    (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (drcov): DRConv2d(
      (conv_kernel): Sequential(
        (0): AdaptiveAvgPool2d(output_size=(3, 3))
        (1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
        (2): Sigmoid()
        (3): Conv2d(64, 32768, kernel_size=(1, 1), stride=(1, 1), groups=8)
      )
      (con

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth


  0%|          | 0.00/83.3M [00:00<?, ?B/s]

Epoch:  0 训练集 loss:  2.6295878887176514
Epoch:  0 测试集 loss:  1.7253327369689941
Epoch:  0  Training complete in 1min 35seconds
Epoch:  0  test acc:  0.34782608695652173
Epoch:  1 训练集 loss:  1.578733205795288
Epoch:  1 测试集 loss:  1.2033778429031372
Epoch:  1  Training complete in 1min 29seconds
Epoch:  1  test acc:  0.4769433465085639
Epoch:  2 训练集 loss:  1.6280854940414429


KeyboardInterrupt: ignored